# <center> 👉 class_10_4 Video » _Object Tracking_ </center>

## Object Tracking 

Simply put, locating an object in successive frames of a video is called tracking.  
The definition sounds straight forward but in computer vision and machine learning, tracking is a very broad term that encompasses conceptually similar but technically different ideas.   
For example, all the following different but related ideas are generally studied under __Object Tracking__  

        간단히 말해서 비디오의 연속 프레임에서 개체를 찾는 것을 추적이라고 합니다.  
        정의는 간단해 보이지만 컴퓨터 비전과 기계 학습에서 추적은 개념적으로 유사하지만 기술적으로 다른 아이디어를 포괄하는 매우 광범위한 용어입니다.   
        예를 들어, 다음과 같지만 관련된 아이디어는 모두 일반적으로 객체 추적에서 연구됩니다
        
* **Dense Optical flow:** These algorithms help estimate the motion vector of every pixel in a video frame.
* **Sparse optical flow:** These algorithms, like the Kanade-Lucas-Tomashi (KLT) feature tracker, track the location of a few feature points in an image.
* **Kalman Filtering:** A very popular signal processing algorithm used to predict the location of a moving object based on prior motion information.   
>- One of the early applications of this algorithm was missile guidance!   
>- Also "the on-board computer that guided the descent of the Apollo 11 lunar module to the moon had a Kalman filter".  
* **Meanshift and Camshift:** These are algorithms for locating the maxima of a density function. They are also used for tracking.
* **Single object trackers:** In this class of trackers, the first frame is marked using a rectangle to indicate the location of the object we want to track.   
The object is then tracked in subsequent frames using the tracking algorithm.   
In most real-life applications, these trackers are used in conjunction with an object detector.
* **Multiple object track finding algorithms:** In cases when we have a fast object detector, it makes sense to detect multiple objects in each frame and then run a track finding algorithm that identifies which rectangle in one frame corresponds to a rectangle in the next frame.
>- Multiple Object Tracking has come a long way.   
>- It uses object detection and novel motion prediction algorithms to get accurate tracking information.   
>- For example, DeepSort uses the YOLO network to get blazing-fast inference speed. It is based on SORT.

## Object tracking using OpenCV 4 – the Tracking API  

OpenCV 4 comes with a tracking API that contains implementations of many single object tracking algorithms.   
There are 8 different trackers available in OpenCV 4.2:
* **BOOSTING, MIL, KCF, TLD, MEDIANFLOW, GOTURN, MOSSE, and CSRT.**   
  
- OpenCV 3.2 has implementations of these 6 trackers — BOOSTING, MIL, TLD, MEDIANFLOW, MOSSE, and GOTURN.   
- OpenCV 3.1 has implementations of these 5 trackers — BOOSTING, MIL, KCF, TLD, MEDIANFLOW.   
- OpenCV 3.0 has implementations of the following 4 trackers — BOOSTING, MIL, TLD, MEDIANFLOW.

## Object tracking using OpenCV – the Algorithms  

Let's begin by first explaining some general principles behind tracking.   
- In tracking, our goal is to find an object in the current frame given we have tracked the object successfully in all (or nearly all) previous frames.  
- Since we have tracked the object up until the current frame, we know how it has been moving.   
- In other words, we know the parameters of the motion model.   
>- The motion model is just a fancy way of saying that you know the location and the velocity (speed + direction of motion) of the object in previous frames.   
>- If you knew nothing else about the object, you could predict the new location based on the current motion model, and you would be pretty close to where the new location of the object is.  
>- But we have more information than just the motion of the object.   
- We know how the object looks in each of the previous frames.   
- In other words, we can build an appearance model that encodes what the object looks like.   
- This appearance model can be used to search in a small neighborhood of the location predicted by the motion model to more accurately predict the location of the object.  
- The motion model predicts the approximate location of the object.   
- The appearance model fine tunes this estimate to provide a more accurate estimate based on appearance.  
- If the object was very simple and did not change it’s appearance much, we could use a simple template as an appearance model and look for that template.   
- However, real life is not that simple. The appearance of an object can change dramatically.   
- To tackle this problem, in many modern trackers, this appearance model is a classifier that is trained in an online manner.   

        먼저 추적의 몇 가지 일반적인 원칙을 설명하는 것부터 시작하겠습니다.
        - 추적에서 우리의 목표는 모든(또는 거의 모든) 이전 프레임에서 객체를 성공적으로 추적한 경우 현재 프레임에서 객체를 찾는 것입니다.
        - 현재 프레임까지 물체를 추적했기 때문에 물체가 어떻게 움직이고 있는지 알 수 있습니다.
        - 즉, 모션 모델의 매개변수를 알고 있습니다.
        >- 모션 모델은 이전 프레임에서 객체의 위치와 속도(속도 + 모션 방향)를 알고 있다는 것을 표현하는 멋진 방법입니다.
        >- 물체에 대해 아무것도 모른다면 현재 모션 모델을 기반으로 새 위치를 예측할 수 있으며 물체의 새 위치에 꽤 가깝습니다.
        >- 하지만 우리는 물체의 움직임보다 더 많은 정보를 갖고 있습니다.
        - 우리는 이전 프레임 각각에서 객체가 어떻게 보이는지 알고 있습니다.
        - 즉, 객체의 모습을 인코딩하는 외관 모델을 구축할 수 있습니다.
        - 이 외관 모델은 모션 모델이 예측한 위치의 작은 주변 지역을 검색하여 물체의 위치를 보다 정확하게 예측하는 데 사용할 수 있습니다.
        - 모션 모델은 물체의 대략적인 위치를 예측합니다.
        - 모양 모델은 이 추정치를 미세 조정하여 모양을 기반으로 보다 정확한 추정치를 제공합니다.
        - 객체가 매우 단순하고 외관이 많이 변하지 않은 경우 간단한 템플릿을 외관 모델로 사용하고 해당 템플릿을 찾을 수 있습니다.
        - 하지만 실제 생활은 그렇게 간단하지 않습니다. 물체의 모양은 극적으로 바뀔 수 있습니다.
        - 이 문제를 해결하기 위해 많은 최신 추적기에서 이 모양 모델은 온라인 방식으로 훈련된 분류자입니다.  
        
The job of the classifier is to classify a rectangular region of an image as either an object or background.   
The classifier takes in an image patch as input and returns a score between 0 and 1 to indicate the probability that the image patch contains the object.   
The score is 0 when it is absolutely sure the image patch is the background and 1 when it is absolutely sure the patch is the object.  

In machine learning, we use the word “online” to refer to algorithms that are trained on the fly at run time.   
An offline classifier may need thousands of examples to train a classifier, but an online classifier is typically trained using very few examples at run time.  

A classifier is trained by feeding it positive (object) and negative (background) examples.   
If you want to build a classifier for detecting cats, you train it with thousands of images containing cats and thousands of images that do not contain cats.   
This way the classifier learns to differentiate what is a cat and what is not.   
While building an online classifier, we do not have the luxury of having thousands of examples of the positive and negative classes.  

        분류자의 역할은 이미지의 직사각형 영역을 객체 또는 배경으로 분류하는 것입니다.
        분류기는 이미지 패치를 입력으로 취하고 0에서 1 사이의 점수를 반환하여 이미지 패치에 객체가 포함될 확률을 나타냅니다.
        이미지 패치가 배경이라고 절대적으로 확신할 때 점수는 0이고, 패치가 객체라고 절대적으로 확신할 때 점수는 1입니다.

        기계 학습에서는 "온라인"이라는 단어를 사용하여 런타임에 즉석에서 훈련되는 알고리즘을 나타냅니다.
        오프라인 분류자는 분류자를 훈련시키기 위해 수천 개의 예가 필요할 수 있지만 온라인 분류자는 일반적으로 런타임에 매우 적은 수의 예를 사용하여 훈련됩니다.

        분류자는 긍정적(객체) 및 부정적(배경) 예시를 제공하여 훈련됩니다.
        고양이를 감지하기 위한 분류기를 구축하려면 고양이가 포함된 수천 개의 이미지와 고양이가 포함되지 않은 수천 개의 이미지로 분류기를 훈련해야 합니다.
        이런 방식으로 분류자는 고양이인지 아닌지를 구별하는 방법을 학습합니다.
        온라인 분류기를 구축하는 동안 우리는 긍정적인 클래스와 부정적인 클래스의 수천 가지 예를 가질 여유가 없습니다.

### ● BOOSTING Tracker  

This tracker is based on an online version of AdaBoost — the algorithm that the HAAR cascade based face detector uses internally. This classifier needs to be trained at runtime with positive and negative examples of the object. The initial bounding box supplied by the user ( or by another object detection algorithm ) is taken as a positive example for the object, and many image patches outside the bounding box are treated as the background.  
Given a new frame, the classifier is run on every pixel in the neighborhood of the previous location and the score of the classifier is recorded. The new location of the object is the one where the score is maximum. So now we have one more positive example for the classifier.   
As more frames come in, the classifier is updated with this additional data.
- Pros: None. This algorithm is a decade old and works ok, but I could not find a good reason to use it especially when other advanced trackers (MIL, KCF) based on similar principles are available.  
- Cons: Tracking performance is mediocre. It does not reliably know when tracking has failed.  
    
        이 추적기는 HAAR 캐스케이드 기반 얼굴 감지기가 내부적으로 사용하는 알고리즘인 AdaBoost의 온라인 버전을 기반으로 합니다. 이 분류자는 객체의 긍정적인 예와 부정적인 예를 사용하여 런타임에 훈련되어야 합니다. 사용자(또는 다른 객체 감지 알고리즘)가 제공한 초기 경계 상자는 객체에 대한 긍정적인 예로 간주되며 경계 상자 외부의 많은 이미지 패치는 배경으로 처리됩니다.
        새로운 프레임이 주어지면 분류기는 이전 위치 근처의 모든 픽셀에서 실행되고 분류기의 점수가 기록됩니다. 개체의 새 위치는 점수가 최대인 위치입니다. 이제 분류기에 대한 긍정적인 예가 하나 더 있습니다.
        더 많은 프레임이 들어오면 분류자는 이 추가 데이터로 업데이트됩니다.
        - 장점: 없음. 이 알고리즘은 10년이 지났고 잘 작동하지만 특히 유사한 원리를 기반으로 하는 다른 고급 추적기(MIL, KCF)를 사용할 수 있는 경우 이 알고리즘을 사용해야 할 타당한 이유를 찾을 수 없습니다.
        - 단점: 추적 성능이 평범합니다. 추적이 언제 실패했는지 확실하게 알 수 없습니다.
            
### ● MIL Tracker  
This tracker is similar in idea to the BOOSTING tracker described above.   
The big difference is that instead of considering only the current location of the object as a positive example, it looks in a small neighborhood around the current location to generate several potential positive examples.   
You may be thinking that it is a bad idea because in most of these “positive” examples the object is not centered.
This is where Multiple Instance Learning (MIL) comes to rescue. In MIL, you do not specify positive and negative examples, but positive and negative “bags”.  
The collection of images in the positive bag are not all positive examples. Instead, only one image in the positive bag needs to be a positive example!  
In our example, a positive bag contains the patch centered on the current location of the object and also patches in a small neighborhood around it.   
Even if the current location of the tracked object is not accurate, when samples from the neighborhood of the current location are put in the positive bag, there is a good chance that this bag contains at least one image in which the object is nicely centered.   
MIL project page has more information for people who like to dig deeper into the inner workings of the MIL tracker.
- Pros: The performance is pretty good. It does not drift as much as the BOOSTING tracker and it does a reasonable job under partial occlusion. If you are using OpenCV 3.0, this might be the best tracker available to you. But if you are using a higher version, consider KCF.
- Cons: Tracking failure is not reported reliably. Does not recover from full occlusion.  
    
        이 추적기는 위에서 설명한 BOOSTING 추적기와 아이디어가 유사합니다.
        가장 큰 차이점은 객체의 현재 위치만을 긍정적인 예로 고려하는 것이 아니라, 현재 위치 주변의 작은 동네를 살펴보며 여러 가지 잠재적인 긍정적인 사례를 생성한다는 점입니다.
        대부분의 "긍정적인" 예에서 객체가 중앙에 있지 않기 때문에 이것이 나쁜 생각이라고 생각할 수도 있습니다.
        이것이 다중 인스턴스 학습(MIL)이 구출되는 곳입니다. MIL에서는 긍정적인 예와 부정적인 예를 지정하는 것이 아니라 긍정적인 예와 부정적인 "백"을 지정합니다.
        포지티브 백에 담긴 이미지 모음이 모두 포지티브 예시는 아닙니다. 대신, 포지티브 백에 있는 이미지 하나만 포지티브 예시가 되어야 합니다!
        이 예에서 포지티브 백에는 객체의 현재 위치를 중심으로 하는 패치와 객체 주변의 작은 이웃에 있는 패치도 포함되어 있습니다.
        추적된 객체의 현재 위치가 정확하지 않더라도 현재 위치 근처의 샘플을 포지티브 백에 넣으면 이 백에 객체가 중앙에 잘 배치된 이미지가 하나 이상 포함될 가능성이 높습니다.
        MIL 프로젝트 페이지에는 MIL 추적기의 내부 작동 방식을 더 깊이 파고드는 사람들을 위한 더 많은 정보가 있습니다.
        - 장점 : 성능이 꽤 좋습니다. BOOSTING 추적기만큼 드리프트하지 않으며 부분 폐쇄 하에서 합리적인 작업을 수행합니다. OpenCV 3.0을 사용하고 있다면 이것이 최고의 트래커일 수 있습니다. 그러나 더 높은 버전을 사용하는 경우 KCF를 고려하십시오.
        - 단점: 추적 실패가 안정적으로 보고되지 않습니다. 완전 폐쇄에서 회복되지 않습니다.  
    
### ● KCF Tracker
KFC stands for Kernelized Correlation Filters. This tracker builds on the ideas presented in the previous two trackers. This tracker utilizes the fact that the multiple positive samples used in the MIL tracker have large overlapping regions. This overlapping data leads to some nice mathematical properties that are exploited by this tracker to make tracking faster and more accurate at the same time.
- Pros: Accuracy and speed are both better than MIL and it reports tracking failure better than BOOSTING and MIL.   
If you are using OpenCV 3.1 and above, I recommend using this for most applications.
- Cons: Does not recover from full occlusion. 
    
        KFC는 커널화된 상관 필터(Kernelized Correlation Filter)를 나타냅니다. 이 추적기는 이전 두 추적기에서 제시된 아이디어를 기반으로 합니다. 이 추적기는 MIL 추적기에서 사용되는 여러 개의 양성 샘플이 큰 중첩 영역을 가지고 있다는 사실을 활용합니다. 이러한 중복되는 데이터는 이 추적기가 활용하여 동시에 더 빠르고 정확한 추적을 가능하게 하는 몇 가지 멋진 수학적 속성으로 이어집니다.
        - 장점: 정확성과 속도 모두 MIL보다 우수하며 BOOSTING 및 MIL보다 추적 실패를 더 잘 보고합니다.
        OpenCV 3.1 이상을 사용하는 경우 대부분의 애플리케이션에 이 버전을 사용하는 것이 좋습니다.
        - 단점: 완전 폐쇄에서 회복되지 않습니다.  
            
### ● TLD Tracker
TLD stands for Tracking, learning, and detection. As the name suggests, this tracker decomposes the long term tracking task into three components — (short term) tracking, learning, and detection. From the author’s paper, “The tracker follows the object from frame to frame. The detector localizes all appearances that have been observed so far and corrects the tracker if necessary.
The learning estimates detector’s errors and updates it to avoid these errors in the future.” This output of this tracker tends to jump around a bit. For example, if you are tracking a pedestrian and there are other pedestrians in the scene, this tracker can sometimes temporarily track a different pedestrian than the one you intended to track. On the positive side, this track appears to track an object over a larger scale, motion, and occlusion. If you have a video sequence where the object is hidden behind another object, this tracker may be a good choice.
- Pros: Works the best under occlusion over multiple frames. Also, tracks best over scale changes.
- Cons: Lots of false positives making it almost unusable.  
    
TLD는 추적(Tracking), 학습(Learning), 탐지(Detection)를 의미합니다. 이름에서 알 수 있듯이 이 추적기는 장기 추적 작업을 (단기) 추적, 학습 및 탐지의 세 가지 구성 요소로 분해합니다. 저자의 논문에서 “추적기는 프레임마다 개체를 따라갑니다. 검출기는 지금까지 관찰된 모든 모양의 위치를 파악하고 필요한 경우 추적기를 수정합니다.
학습은 감지기의 오류를 추정하고 이를 업데이트하여 향후 이러한 오류를 방지합니다." 이 추적기의 출력은 약간 점프하는 경향이 있습니다. 예를 들어, 보행자를 추적하고 있는데 장면에 다른 보행자가 있는 경우 이 추적기는 때때로 추적하려는 보행자가 아닌 다른 보행자를 일시적으로 추적할 수 있습니다. 긍정적인 측면에서 이 트랙은 더 큰 규모, 모션 및 폐색에 걸쳐 개체를 추적하는 것으로 보입니다. 개체가 다른 개체 뒤에 숨겨져 있는 비디오 시퀀스가 있는 경우 이 추적기가 좋은 선택일 수 있습니다.
- 장점: 여러 프레임에 걸쳐 폐색된 상태에서 가장 잘 작동합니다. 또한 규모 변화에 대해 가장 잘 추적합니다.
- 단점: 오탐(false positive)이 많아 거의 사용할 수 없습니다.  
    
### ● MEDIANFLOW Tracker
Internally, this tracker tracks the object in both forward and backward directions in time and measures the discrepancies between these two trajectories.   
Minimizing this ForwardBackward error enables them to reliably detect tracking failures and select reliable trajectories in video sequences.
In my tests, I found this tracker works best when the motion is predictable and small.   
Unlike, other trackers that keep going even when the tracking has clearly failed, this tracker knows when the tracking has failed.
- Pros: Excellent tracking failure reporting. Works very well when the motion is predictable and there is no occlusion.  
- Cons: Fails under large motion.  
    
        내부적으로 이 추적기는 시간에 따라 앞뒤 방향으로 물체를 추적하고 이 두 궤적 사이의 불일치를 측정합니다.
        이 ForwardBackward 오류를 최소화하면 추적 실패를 안정적으로 감지하고 비디오 시퀀스에서 신뢰할 수 있는 궤적을 선택할 수 있습니다.
        테스트에서 이 추적기는 움직임이 예측 가능하고 작을 때 가장 잘 작동하는 것으로 나타났습니다.
        추적이 확실히 실패했음에도 계속 작동하는 다른 추적기와 달리 이 추적기는 추적이 실패한 시점을 알 수 있습니다.
        - 장점: 추적 실패 보고가 뛰어납니다. 모션이 예측 가능하고 폐색이 없을 때 매우 잘 작동합니다.
        - 단점: 큰 움직임에서는 실패합니다.  
            
### ● GOTURN tracker
Out of all the tracking algorithms in the tracker class, this is the only one based on Convolutional Neural Network (CNN).   
From OpenCV documentation, we know it is “robust to viewpoint changes, lighting changes, and deformations”.   
But it does not handle occlusion very well.
* GOTURN being a CNN based tracker, uses a Caffe model for tracking.   
* The Caffe model and the proto text file must be present in the directory in which the code is present.   
* These files can also be downloaded from the opencv_extra repository, concatenated, and extracted before use.
* Update: GOTURN object tracking algorithm has been ported to OpenCV.

### ● MOSSE tracker
- Minimum Output Sum of Squared Error (MOSSE) uses an adaptive correlation for object tracking which produces stable correlation filters when initialized using a single frame.   
- MOSSE tracker is robust to variations in lighting, scale, pose, and non-rigid deformations.   
- It also detects occlusion based upon the peak-to-sidelobe ratio, which enables the tracker to pause and resume where it left off when the object reappears.   
- MOSSE tracker also operates at a higher fps (450 fps and even more).   
- To add to the positives, it is also very easy to implement, is as accurate as other complex trackers and much faster.   
- But, on a performance scale, it lags behind the deep learning based trackers.  

        - MOSSE(Minimum Output Sum of Squared Error)는 단일 프레임을 사용하여 초기화할 때 안정적인 상관 필터를 생성하는 개체 추적을 위한 적응형 상관 관계를 사용합니다.
        - MOSSE 추적기는 조명, 규모, 포즈 및 비강성 변형의 변화에 강력합니다.
        - 또한 피크 대 사이드로브 비율을 기반으로 폐색을 감지하므로 객체가 다시 나타날 때 추적기가 중단된 부분부터 일시 중지하고 다시 시작할 수 있습니다.
        - MOSSE 추적기는 더 높은 fps(450fps 이상)에서도 작동합니다.
        - 장점을 추가하자면, 구현하기가 매우 쉽고, 다른 복잡한 추적기만큼 정확하고 훨씬 빠릅니다.
        - 하지만 성능면에서는 딥러닝 기반의 트래커에 비해 뒤떨어집니다.  
        
### ● CSRT tracker
- In the Discriminative Correlation Filter with Channel and Spatial Reliability (DCF-CSR), we use the spatial reliability map for adjusting the filter support to the part of the selected region from the frame for tracking.   
- This ensures enlarging and localization of the selected region and improved tracking of the non-rectangular regions or objects.   
- It uses only 2 standard features (HoGs and Colornames).   
- It also operates at a comparatively lower fps (25 fps) but gives higher accuracy for object tracking.   

        - DCF-CSR(Discriminative Correlation Filter with Channel and Spatial Reliability)에서는 추적을 위해 프레임에서 선택한 영역의 일부에 대한 필터 지원을 조정하기 위해 공간 신뢰도 맵을 사용합니다.
        - 이를 통해 선택한 영역의 확대 및 위치화가 보장되고 직사각형이 아닌 영역이나 개체에 대한 추적이 향상됩니다.
        - 2가지 표준 기능(HoG 및 Colornames)만 사용합니다.
        - 상대적으로 낮은 fps(25fps)에서도 작동하지만 객체 추적에 있어 더 높은 정확도를 제공합니다.      
    
In the commented code below we first set up the tracker by choosing a tracker type   
- BOOSTING, MIL, KCF, TLD, MEDIANFLOW, GOTURN, MOSSE, or CSRT.   
We then open a video and grab a frame.   
We define a bounding box containing the object for the first frame and initialize the tracker with the first frame and the bounding box.   
Finally, we read frames from the video and just update the tracker in a loop to obtain a new bounding box for the current frame.  
Results are subsequently displayed.

In [ ]:
## Tracking API  

OpenCV provides a Tracking API for tracking objects.   
The Tracking API makes it easy to track objects.   
The Tracking API constructor provided by OpenCV is shown below.   
The constructor varies depending on the algorithm.  

        OpenCV에서는 객체 추적을 위한 Tracking API를 제공합니다.   
        Tracking API를 이용하면 쉽게 객체 추적을 할 수 있습니다.   
        OpenCV에서 제공하는 Tracking API생성자는 아래와 같습니다.   
        생성자는 알고리즘에 따라 다양합니다.

* **tracker = cv2.TrackerBoosting_create():** Based on the AdaBoost algorithm
* **tracker = cv2.TrackerMIL_create():** Based on the MIL(Multiple Instance Learning) algorithm
* **tracker = cv2.TrackerKCF_create():** Based on KCF(Kernelized Correlation Filters) algorithm
* **tracker = cv2.TrackerTLD_create():** Based on TLD(Tracking, Learning and Detection) algorithm
* **tracker = cv2.TrackerMedianFlow_create():** Track the forward and reverse directions of objects to measure inconsistencies 객체의 전방향/역방향을 추적해서 불일치성을 측정
* tracker = cv2.TrackerGOTURN_cretae(): Based on CNN(Convolutional Neural Networks)  not work
* tracker = cv2.TrackerCSRT_create(): Based on CSRT(Channel and Spatial Reliability) not work
* tracker = cv2.TrackerMOSSE_create(): Use grayscale internally not work

- retval = cv2.Tracker.init(img, boundingBox): initialize Tracker 
>- img: input
>- boundingBox: roi (x, y)

- retval, boundingBox = cv2.Tracker.update(img): Find the location of the object to be tracked in a new frame
>- img: new frame
>- retval: track boolean
>- boundingBox: new roi (x, y, w, h)


The first time you run the code, the screen will freeze.   
- First, drag the object you want to track.   
- Next, press the Space key. It will then play back on its own, and start tracking the object.   
- You can press the keyboard 0~4 number keys to select the tracker algorithm.   
- At the top of the screen, you will see 0:TrackerBoosting, 1:TrackerMIL, etc.   
- This is an indication of which tracker you are currently using to track an object.

코드를 처음 실행하면 화면이 멈춰있을 겁니다.   
- 먼저 추적을 원하는 객체를 드래그합니다.   
- 다음으로 스페이스(space) 키를 누릅니다. 그러면 알아서 재생되고, 객체 추적을 시작합니다.   
- 키보드 0~4 숫자 키를 눌러 트랙커 알고리즘을 선택할 수 있습니다.   
- 화면 상단의 0:TrackerBoosting, 1:TrackerMIL 등이 뜰 겁니다.   
- 현재 어떤 트랙커로 객체 추적을 하고 있는지를 나타내는 표시입니다.

In [ ]:
# Tracker APIs (track_trackingAPI.py)

import cv2

trackers = [cv2.TrackerBoosting_create,  # trackers idx no = 0
            cv2.TrackerMIL_create,       # trackers idx no = 1
            cv2.TrackerKCF_create,       # trackers idx no = 2
            cv2.TrackerTLD_create,       # trackers idx no = 3
            cv2.TrackerMedianFlow_create,# trackers idx no = 4
#             cv2.TrackerGOTURN_create, # not work
#             cv2.TrackerCSRT_create,   # not work
#             cv2.TrackerMOSSE_create   # not work
           ]
trackerIdx = 0     # trackers idx no
tracker = None
isFirst = True

video_src_f = './Videos/car_drive.mp4'
cap = cv2.VideoCapture(video_src_f)
fps = cap.get(cv2.CAP_PROP_FPS) 
delay = int(1000/fps)
win_name = 'Tracking APIs'

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print('Cannot read video file')
        break
    img_draw = frame.copy()
    if tracker is None:            
        cv2.putText(img_draw, "Press the Space to set ROI!!", \
            (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2,cv2.LINE_AA)
    else:
        ok, bbox = tracker.update(frame)   # set roi
        (x,y,w,h) = bbox
        if ok:                             # track
            cv2.rectangle(img_draw, (int(x), int(y)), (int(x + w), int(y + h)), (0,255,0), 2, 1)
        else : 
            cv2.putText(img_draw, "Tracking fail.", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2,cv2.LINE_AA)
    trackerName = tracker.__class__.__name__
    cv2.putText(img_draw, str(trackerIdx) + ":"+trackerName , (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0),2,cv2.LINE_AA)
    cv2.imshow(win_name, img_draw)
    key = cv2.waitKey(delay) & 0xff
   
    if key == ord(' ') or (video_src_f != 0 and isFirst):  # space bar
        isFirst = False
        roi = cv2.selectROI(win_name, frame, False)        # set roi
        if roi[2] and roi[3]:        
            tracker = trackers[trackerIdx]()    
            isInit = tracker.init(frame, roi)
    elif key in range(48, 56):                             # if tracker idx has changeed input 0~4
        trackerIdx = key-48     
        if bbox is not None:
            tracker = trackers[trackerIdx]() 
            isInit = tracker.init(frame, bbox) 
    elif key == 27 : 
        break
# else:
#     print( "Could not open video")
    
cap.release()
cv2.destroyAllWindows()

# ▣ Single Object Trackers

In [1]:
import cv2

def ask_for_tracker():
    print('Hi,! What tracker API would you like to use? ')
    print('Enter 0 for BOOSTING')
    print('Enter 1 for MIL')
    print('Enter 2 for KCF')
    print('Enter 3 for TLD')
    print('Enter 4 for MEDIANFLOW')
    print('Enter 5 for GOTURN')
    print('Enter 6 for MOSSE')
    print('Enter 7 for CSRT')
    
    choise = input('Enter your tracker ')
    
    if choise == '0':
        tracker = cv2.TrackerBoosting_create()
    elif choise == '1':
        tracker = cv2.TrackerMIL_create()
    elif choise == '2':
        tracker = cv2.TrackerKCF_create()
    elif choise == '3':
        tracker = cv2.TrackerTLD_create()
    elif choise == '4':
        tracker = cv2.TrackerMedianFlow_create()
    elif choise == '5':
        tracker = cv2.TrackerGOTURN_create()
    elif choise == '6':
        tracker = cv2.TrackerMOSSE_create()
    elif choise == '7':
        tracker = cv2.TrackerCSRT_create()
        
    return tracker

tracker = ask_for_tracker()            # Tracker
tracker_name = str(tracker).split()[0][1:]  # Tracker name
cap = cv2.VideoCapture('./Videos/Vehicles.mp4')

ret, frame = cap.read()
roi = cv2.selectROI(frame, False)
ret = tracker.init(frame, roi)

while True:
    ret, frame = cap.read()
    success, roi = tracker.update(frame)  # update tracker
    (x, y, w, h) = tuple(map(int, roi))   # roi -> from tuple to int
    cv2.putText(frame, str(tracker), (100,120), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0),2,cv2.LINE_AA)
    
    if success:                           # Sucess on tracking
        pts1 = (x,y)
        pts2 = (x+w, y+h)
        cv2.rectangle(frame, pts1, pts2, (255, 125, 25), 3)  # Draw rects as tracker moves
        
    else:                      # Failure on tracking  
        cv2.putText(frame, 'Fail to trace the object',(100, 100),cv2.FONT_HERSHEY_SIMPLEX, 1, (25, 125, 255),3)
    
    cv2.putText(frame,tracker_name,(20, 400),cv2.FONT_HERSHEY_SIMPLEX, 1, (225, 225, 0), 3)  # Display Tracker
    cv2.imshow(tracker_name, frame)

    if cv2.waitKey(50) & 0xFF == 27:
        break  

cap.release()
cv2.destroyAllWindows()

Hi,! What tracker API would you like to use? 
Enter 0 for BOOSTING
Enter 1 for MIL
Enter 2 for KCF
Enter 3 for TLD
Enter 4 for MEDIANFLOW
Enter 5 for GOTURN
Enter 6 for MOSSE
Enter 7 for CSRT
Enter your tracker 4


# ▣ Multi Object Trackers

In [2]:
import cv2
import sys
from random import randint
tracker_types = ['BOOSTING',
                'MIL',
                'KCF',
                'TLD',
                'MEADIANFLOW',
                'GOTURN',
                'MOSSE',
                'CSRT']

def tracker_name(tracker_type):

    if tracker_type == tracker_types[0]:
        tracker = cv2.TrackerBoosting_create()
    elif tracker_type == tracker_types[1]:
        tracker = cv2.TrackerMIL_create()
    elif tracker_type == tracker_types[2]:
        tracker = cv2.TrackerKCF_create()
    elif tracker_type == tracker_types[3]:
        tracker = cv2.TrackerTLD_create()
    elif tracker_type == tracker_types[4]:
        tracker = cv2.TrackerMedianFlow_create()
    elif tracker_type == tracker_types[5]:
        tracker = cv2.TrackerGOTURN_create()
    elif tracker_type == tracker_types[6]:
        tracker = cv2.TrackerMOSSE_create()
    elif tracker_type == tracker_types[7]:
        tracker = cv2.TrackerCSRT_create()

    else:
        tracker = None
        print('No tracker found')
        print('Choose from this trackers: ')
        for tr in tracker_types:
            print(tr)
                                    
    return tracker        

if __name__ == '__main__':
    print("Default tracking algorithm 'KCF' \n" "Available algorithms are: \n")
    for ta in tracker_types:
        print(ta)
    tracker_type = 'KCF'
    cap = cv2.VideoCapture('./Videos/Vehicles.mp4')
    success, frame = cap.read()
    if not success:
        print('Cannot read the video')
    rects = []
    colors = []
    
    while True:
    
        rect_box = cv2.selectROI('Multitracker', frame)
        rects.append(rect_box)
        colors.append((randint(64, 255), randint(64,255), randint(64,255)))
        print('Press q to stop selecting boxes and start multitracking')
        print('Press any key to select another box')

        if cv2.waitKey(0) & 0xFF == 27:
            break
    print(f'Selected boxes {rects}')
    multitracker = cv2.MultiTracker_create()  # Create multitracker
    
    for rect_box in rects:                    # Initialize multitracker
        multitracker.add(tracker_name(tracker_type),frame,rect_box)
    
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        success, boxes = multitracker.update(frame)  # update location objects
        
        for i, newbox in enumerate(boxes):    # draw the objectes tracked
            pts1 = (int(newbox[0]), int(newbox[1]))
            pts2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
            cv2.rectangle(frame, pts1, pts2, colors[i], 2, 1)
        
        cv2.imshow('Multitracker', frame)
        
        if cv2.waitKey(20) & 0xFF == 27:
            break    
    
cap.release()
cv2.destroyAllWindows()

Default tracking algorithm 'KCF' 
Available algorithms are: 

BOOSTING
MIL
KCF
TLD
MEADIANFLOW
GOTURN
MOSSE
CSRT
Press q to stop selecting boxes and start multitracking
Press any key to select another box
Selected boxes [(787, 389, 79, 75)]
